### Seq2Seq
---

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import TranslationDataset, Multi30k, IWSLT
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [2]:
SEED = 11747

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [4]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [5]:
SRC = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TGT = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [6]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.en', '.de'), 
                                                    fields = (SRC, TGT))

In [7]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


In [8]:
print(vars(train_data.examples[0]))

{'src': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.'], 'trg': ['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'im', 'sind', 'männer', 'weiße', 'junge', 'zwei']}


In [9]:
SRC.build_vocab(train_data, min_freq = 2)
TGT.build_vocab(train_data, min_freq = 2)

print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TGT.vocab)}")

Unique tokens in source (de) vocabulary: 5893
Unique tokens in target (en) vocabulary: 7855


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

#### Simple LSTM Encoder Decoders

In [12]:
class Attention(nn.Module):
    def __init__(self, enc_nhid, dec_nhid):
        super(Attention, self).__init__()
        self.attn = nn.Linear(enc_nhid*2+dec_nhid, dec_nhid)
        self.v = nn.Linear(dec_nhid, 1, bias=False)
        
    def forward(self, hidden, enc_out):
        # hidden: (2, b, nhid)
        # enc_out: (s, b, nhid*2)
        b = enc_out.shape[1]
        src_len = enc_out.shape[0]
        hidden = hidden[-1].unsqueeze(1).repeat(1, src_len, 1)
        enc_out = enc_out.permute(1, 0, 2)
        energy = torch.tanh(self.attn(torch.cat((hidden, enc_out), dim=2)))
        # energy: (b, s, nhid)
        attn = self.v(energy).squeeze(2)
        return F.softmax(attn, dim=1)

In [21]:
class Encoder(nn.Module):
    def __init__(self, ninp, nembed, enc_nhid, dec_nhid, nlayers, dropout=0.2):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(ninp, nembed)
        self.nlayers = nlayers
        self.rnn = nn.GRU(nembed, enc_nhid, nlayers, dropout=dropout, bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(enc_nhid*2, dec_nhid)
        
    def forward(self, src):
        # src: (s, b)
        s, b = src.shape
        src = self.dropout(self.embedding(src))
        # src: (s, b, nembed), hidden: (nlayers*dir, b, nhid), cell: (nlayers*dir, b, nhid)
        out, hidden = self.rnn(src)
        hidden = hidden.transpose(1,0).reshape(b, self.nlayers, -1).transpose(1,0)
        hidden = torch.tanh(self.fc(hidden))
        return out, hidden
    
class Decoder(nn.Module):
    def __init__(self, nout, nembed, enc_nhid, dec_nhid, nlayers, dropout=0.2, attention=None):
        super(Decoder, self).__init__()
        self.nout = nout
        self.nlayers = nlayers
        self.embedding = nn.Embedding(nout, nembed)
        self.rnn = nn.GRU(nembed+enc_nhid*2, dec_nhid, nlayers, 
                          dropout=dropout, bidirectional=False)
        self.fc_out = nn.Linear(enc_nhid*2+dec_nhid+nembed, nout)
        self.dropout = nn.Dropout(dropout)
        self.attention = attention
    
    def forward(self, src, hidden, enc_out):
        """
        for decoder, we process one token at a time
        """
        # src: (b, )
        embedded = src.unsqueeze(0)
        embedded = self.dropout(self.embedding(embedded))
        attn = self.attention(hidden, enc_out)
        attn = attn.unsqueeze(1)
        enc_out = enc_out.permute(1,0,2)
        ctxt = torch.bmm(attn, enc_out).permute(1,0,2)
        x = torch.cat((embedded, ctxt), dim=2)
        out, hidden = self.rnn(x, hidden)
        # out, hidden: (1, b, nhid)
        
        embedded = embedded.squeeze(0)
        out = out.squeeze(0)
        ctxt = ctxt.squeeze(0)
        pred = self.fc_out(torch.cat((out, ctxt, embedded), dim=1))
        return pred, hidden
        

In [22]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, tgt, teacher_forcing_ratio=0.5):
        # src: (src_len, b)
        # tgt: (tgt_len, b)
        tgt_len, b = tgt.shape
        tgt_vocab_size = self.decoder.nout
        
        # tensor to store decoder outputs
        outs = torch.zeros(tgt_len, b, tgt_vocab_size).to(self.device)
        
        enc_out, hidden = self.encoder(src)
        x = tgt[0]
        for t in range(1, tgt_len):
            out, hidden = self.decoder(x, hidden, enc_out)
            outs[t] = out
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = out.argmax(1)
            x = tgt[t] if teacher_force else top1
            
        return outs

In [23]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TGT.vocab)
ENC_EMB_DIM = 250
DEC_EMB_DIM = 250
ENC_HID_DIM = 500
DEC_HID_DIM = 500
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
NLAYERS = 2

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, NLAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, NLAYERS, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [24]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(5893, 250)
    (rnn): GRU(250, 500, num_layers=2, dropout=0.5, bidirectional=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (fc): Linear(in_features=1000, out_features=500, bias=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(7855, 250)
    (rnn): GRU(1250, 500, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=1750, out_features=7855, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (attention): Attention(
      (attn): Linear(in_features=1500, out_features=500, bias=True)
      (v): Linear(in_features=500, out_features=1, bias=False)
    )
  )
)

In [25]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 29,335,605 trainable parameters


In [26]:
optimizer = optim.Adam(model.parameters())
TGT_PAD_IDX = TGT.vocab.stoi[TGT.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TGT_PAD_IDX)

In [27]:
def train_epoch(model, iterator, optimizer, criterion, clip):
    model.train()
    train_loss = 0
    for i, batch in enumerate(iterator):
        src = batch.src
        tgt = batch.trg
        optimizer.zero_grad()
        out = model(src, tgt)
        out_dim = out.shape[-1]
        out = out[1:].view(-1, out_dim)
        tgt = tgt[1:].view(-1)
        
        loss = criterion(out, tgt)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        train_loss += loss.item()
    return train_loss/len(iterator)


def evaluate(model, iterator, criterion):
    model.eval()
    loss = 0
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch.src
            tgt = batch.trg
            out = model(src, tgt, 0) # no teacher forcing here
            out_dim = out.shape[-1]
            out = out[1:].view(-1, out_dim)
            tgt = tgt[1:].view(-1)
            loss += criterion(out, tgt).item()
    return loss/len(iterator)


def epoch_time(start, end):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [28]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss = train_epoch(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

RuntimeError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 7.77 GiB total capacity; 2.55 GiB already allocated; 18.88 MiB free; 2.66 GiB reserved in total by PyTorch)